In [1]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

num_classes = 4

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Add your own layers if needed
predictions = Dense(num_classes, activation='softmax')(x)  # Adjust num_classes according to your dataset

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [3]:
import os
import pandas as pd
import numpy as np
import random
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define function to randomly sample a subset of data
def random_sample(root_dir, csv_filename, sample_size):
    data = pd.read_csv(csv_filename)
    sampled_data = data.sample(n=sample_size, random_state=42)  # Adjust sample_size as needed
    sampled_images = []
    sampled_labels = []
    for index, row in sampled_data.iterrows():
        image_path = os.path.join(root_dir, row['image'])
        image = load_img(image_path, target_size=(200, 200))
        image = img_to_array(image) / 255.0
        sampled_images.append(image)
        sampled_labels.append([int(x) for x in row['label'].split()])  # Convert string label to list of integers
    return sampled_images, sampled_labels

# Randomly sample data for 2x2 puzzle
train_images_2x2, train_labels_2x2 = random_sample('../Dataset/puzzle_2x2/train', '../Dataset/puzzle_2x2/train.csv', 9000)
test_images_2x2, test_labels_2x2 = random_sample('../Dataset/puzzle_2x2/test', '../Dataset/puzzle_2x2/test.csv', 2176)
valid_images_2x2, valid_labels_2x2 = random_sample('../Dataset/puzzle_2x2/valid', '../Dataset/puzzle_2x2/valid.csv', 385)

# # Randomly sample data for 3x3 puzzle
# train_images_3x3, train_labels_3x3 = random_sample('../Dataset/puzzle_3x3/train', '../Dataset/puzzle_3x3/train.csv', 100)
# test_images_3x3, test_labels_3x3 = random_sample('../Dataset/puzzle_3x3/test', '../Dataset/puzzle_3x3/test.csv', 20)
# valid_images_3x3, valid_labels_3x3 = random_sample('../Dataset/puzzle_3x3/valid', '../Dataset/puzzle_3x3/valid.csv', 20)

# # Concatenate 2x2 and 3x3 datasets
# train_images = train_images_2x2 + train_images_3x3
# train_labels = train_labels_2x2 + train_labels_3x3
# test_images = test_images_2x2 + test_images_3x3
# test_labels = test_labels_2x2 + test_labels_3x3
# valid_images = valid_images_2x2 + valid_images_3x3
# valid_labels = valid_labels_2x2 + valid_labels_3x3

# Convert lists to numpy arrays
train_data = np.array(train_images_2x2)
test_data = np.array(test_images_2x2)
valid_data = np.array(valid_images_2x2)
train_labels = np.array(train_labels_2x2)
test_labels = np.array(test_labels_2x2)
valid_labels = np.array(valid_labels_2x2)

# Print shapes of the datasets
print("Train data shape:", train_data.shape)
print("Train labels shape:", train_labels.shape)
print("Test data shape:", test_data.shape)
print("Test labels shape:", test_labels.shape)
print("Validation data shape:", valid_data.shape)
print("Validation labels shape:", valid_labels.shape)


Train data shape: (9000, 200, 200, 3)
Train labels shape: (9000, 4)
Test data shape: (2176, 200, 200, 3)
Test labels shape: (2176, 4)
Validation data shape: (385, 200, 200, 3)
Validation labels shape: (385, 4)


In [4]:
# Train the model
model.fit(train_data, train_labels, validation_data=(valid_data, valid_labels), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(test_data, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/10
282/282 [==============================] - 950s 3s/step - loss: 72.9703 - accuracy: 0.2440 - val_loss: 64.2262 - val_accuracy: 0.2312
Epoch 2/10
282/282 [==============================] - 885s 3s/step - loss: 81.0716 - accuracy: 0.2414 - val_loss: 60.1680 - val_accuracy: 0.2494
Epoch 3/10
282/282 [==============================] - 1018s 4s/step - loss: 158.6624 - accuracy: 0.2539 - val_loss: 87.1152 - val_accuracy: 0.2312
Epoch 4/10
282/282 [==============================] - 1175s 4s/step - loss: 186.1860 - accuracy: 0.2471 - val_loss: 164.3188 - val_accuracy: 0.2494
Epoch 5/10
282/282 [==============================] - 1069s 4s/step - loss: 192.9041 - accuracy: 0.2456 - val_loss: 234.9616 - val_accuracy: 0.2753
Epoch 6/10
282/282 [==============================] - 861s 3s/step - loss: 198.7936 - accuracy: 0.2416 - val_loss: 547.3972 - val_accuracy: 0.2494
Epoch 7/10
282/282 [==============================] - 849s 3s/step - loss: 218.2659 - accuracy: 0.2587 - val_loss: 345.8